<a href="https://colab.research.google.com/github/JhonysIV/SenalesAndSistemas/blob/main/Parcial_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Ejercicio 1

In [39]:
# ═══════════════════════════════════════════════════════════════════════════
# IMPORTS - Librerías

import os
import numpy as np
import librosa
from scipy.io.wavfile import write
from scipy.fft import fft, ifft, fftfreq, fftshift
import matplotlib.pyplot as plt
# ═══════════════════════════════════════════════════════════════════════════

# ═══════════════════════════════════════════════════════════════════════════
# CONFIGURACIÓN DEL SISTEMA DSB-SC

AUDIO_FILENAME = 'Torero.mp3'
START_TIME_S = 108          # Minuto 1:48 (108 segundos)
DURATION_S = 5              # Duración 5 segundos

# Parámetros de Modulación
CARRIER_FREQ = 10000        # f0: Frecuencia de portadora (10 kHz)
AUDIO_BW = 4000             # Ancho de banda del mensaje (Limitamos a 4kHz para evitar Aliasing)
# ═══════════════════════════════════════════════════════════════════════════

# --- Descargar audio si no existe ---
if not os.path.exists(AUDIO_FILENAME):
    print(f"'{AUDIO_FILENAME}' no encontrado. Descargando...")
    # !!! REEMPLAZA ESTA URL CON TU ENLACE PÚBLICO AL ARCHIVO 'Torero.mp3' !!!
    !wget -q "https://raw.githubusercontent.com/JhonysIV/SenalesAndSistemas/main/Parcial_2/Torero.mp3" -O {AUDIO_FILENAME}
    print(f"'{AUDIO_FILENAME}' descargado.")
else:
    print(f"'{AUDIO_FILENAME}' ya existe.")
# ═══════════════════════════════════════════════════════════════════════════

# ═══════════════════════════════════════════════════════════════════════════
# HERRAMIENTAS AUXILIARES

def verificar_archivo_local(filename):
    if os.path.exists(filename):
        return True
    else:
        return False

def guardar_wav(filename, signal, sr):
    # Normalizar para evitar saturación al guardar
    max_val = np.max(np.abs(signal))
    if max_val > 0:
        signal_norm = signal / max_val
    else:
        signal_norm = signal
    write(filename, sr, np.int16(signal_norm * 32767))
# ═══════════════════════════════════════════════════════════════════════════

# ═══════════════════════════════════════════════════════════════════════════
# VISUALIZACIÓN (TIEMPO Y FRECUENCIA)

def graficar_etapas(t, signals, titulos, sr):
    n_etapas = len(signals)
    fig, axes = plt.subplots(n_etapas, 2, figsize=(16, 12))

    for i, (sig, titulo) in enumerate(zip(signals, titulos)):
        # 1. Dominio del Tiempo (Zoom a un fragmento pequeño para ver la onda)
        ax_time = axes[i, 0]
        zoom_samples = int(0.01 * sr)
        ax_time.plot(t[:zoom_samples], sig[:zoom_samples], color='tab:blue')
        ax_time.set_title(f"{titulo} (Dominio Tiempo - Zoom)", fontsize=10)
        ax_time.set_ylabel("Amplitud")
        ax_time.grid(alpha=0.3)

        # 2. Dominio de la Frecuencia (Espectro)
        ax_freq = axes[i, 1]
        N = len(sig)
        yf = fft(sig)
        xf = fftfreq(N, 1 / sr)

        half_N = N // 2
        ax_freq.plot(xf[:half_N], 2.0/N * np.abs(yf[:half_N]), color='tab:orange')
        ax_freq.set_title(f"{titulo} (Espectro Frecuencia)", fontsize=10)
        ax_freq.set_xlabel("Frecuencia (Hz)")
        ax_freq.set_xlim(0, 20000) # Ver hasta 20kHz
        ax_freq.grid(alpha=0.3)

    plt.tight_layout()
    return fig
# ═══════════════════════════════════════════════════════════════════════════

# El bloque de ejecución principal ha sido movido a app.py para la interfaz de Streamlit.

'Torero.mp3' no encontrado. Descargando...
'Torero.mp3' descargado.


Ejercicio 2

In [40]:
# Instalación de librerías necesarias
# El módulo 'control' a menudo facilita la manipulación de FT y Lazo Cerrado, además de SciPy.
!pip install streamlit sympy scipy numpy matplotlib control librosa -q

In [41]:
%%writefile app.py




import streamlit as st
import sympy as sym
from sympy.physics.control.lti import TransferFunction
from sympy.physics.control.control_plots import pole_zero_plot
from sympy.integrals.transforms import inverse_laplace_transform # Import for sympy.inverse_laplace_transform
from scipy import signal
import numpy as np
import matplotlib.pyplot as plt
import control as ct

# --- DSP Specific Imports (moved from original DSP cell) ---
import os
import librosa
from scipy.io.wavfile import write
from scipy.fft import fft, ifft, fftfreq, fftshift

# --- CONFIGURACIÓN DE STREAMLIT ---
st.set_page_config(layout="wide")
st.title("👨‍🔬 Análisis del Sistema Masa-Resorte-Amortiguador")
st.markdown("---")

# --- DEFINICIÓN GLOBAL DE VARIABLES SYMPY ---
s = sym.Symbol('s')

# --- DEFINICIÓN DE PARÁMETROS BÁSICOS ---
m_base = 1
k_base = 4
wn_base = np.sqrt(k_base / m_base)
c_critico = 2 * np.sqrt(m_base * k_base)

# --- ARREGLO PARA LA ITERACIÓN DE CASOS ---
CASOS = {
    "Subamortiguado (ζ=0.5)": {'c': c_critico * 0.5, 'zeta_target': 0.5},
    "Crítico (ζ=1)": {'c': c_critico, 'zeta_target': 1.0},
    "Sobreamortiguado (ζ=1.25)": {'c': c_critico * 1.25, 'zeta_target': 1.25},
}

# --- DSP HELPER FUNCTIONS (moved from original DSP cell) ---
def verificar_archivo_local(filename):
    if os.path.exists(filename):
        return True
    else:
        return False

def guardar_wav(filename, signal, sr):
    max_val = np.max(np.abs(signal))
    if max_val > 0:
        signal_norm = signal / max_val
    else:
        signal_norm = signal
    write(filename, sr, np.int16(signal_norm * 32767))

def graficar_etapas(t, signals, titulos, sr):
    n_etapas = len(signals)
    fig, axes = plt.subplots(n_etapas, 2, figsize=(16, 12))

    for i, (sig, titulo) in enumerate(zip(signals, titulos)):
        ax_time = axes[i, 0]
        zoom_samples = int(0.01 * sr)
        ax_time.plot(t[:zoom_samples], sig[:zoom_samples], color='tab:blue')
        ax_time.set_title(f"{titulo} (Dominio Tiempo - Zoom)", fontsize=10)
        ax_time.set_ylabel("Amplitud")
        ax_time.grid(alpha=0.3)

        ax_freq = axes[i, 1]
        N = len(sig)
        yf = fft(sig)
        xf = fftfreq(N, 1 / sr)

        half_N = N // 2
        ax_freq.plot(xf[:half_N], 2.0/N * np.abs(yf[:half_N]), color='tab:orange')
        ax_freq.set_title(f"{titulo} (Espectro Frecuencia)", fontsize=10)
        ax_freq.set_xlabel("Frecuencia (Hz)")
        ax_freq.set_xlim(0, 20000)
        ax_freq.grid(alpha=0.3)

    plt.tight_layout()
    return fig


# --- PROCESO PRINCIPAL: MODULACIÓN Y DEMODULACIÓN DSB-SC (Adaptado para Streamlit) ---
def procesar_dsp_streamlit(audio_filename, start_time_s, duration_s, carrier_freq, audio_bw):
    if not verificar_archivo_local(audio_filename):
        st.error(f"ERROR: No se encuentra '{audio_filename}'. Asegúrate de que esté subido al entorno de Colab.")
        return None, None

    st.write(f"Cargando audio '{audio_filename}'...")
    y, sr = librosa.load(audio_filename, sr=None, offset=start_time_s, duration=duration_s)

    N = len(y)
    yf_orig = fft(y)
    xf = fftfreq(N, 1/sr)
    mask_bw = np.abs(xf) < audio_bw
    y = np.real(ifft(yf_orig * mask_bw))
    y = y / np.max(np.abs(y))

    t = np.linspace(0, duration_s, N, endpoint=False)
    carrier = np.cos(2 * np.pi * carrier_freq * t)

    # ETAPA 1: MODULACIÓN
    x_t = y * carrier

    # ETAPA 2: MEZCLADOR
    y_mix = x_t * carrier

    # ETAPA 3: FILTRO PASA BAJAS
    Y_f = fft(y_mix)
    cutoff_lpf = audio_bw + 500
    mask_lpf = np.abs(xf) < cutoff_lpf
    Z_f = Y_f * mask_lpf
    z_t = np.real(ifft(Z_f))

    # ETAPA 4: ESCALADO DE AMPLITUD
    m_recovered = z_t * 2

    signals = [y, x_t, y_mix, m_recovered]
    titulos = [
        "1. Mensaje Original m(t)",
        f"2. Señal Modulada DSB-SC (Portadora {carrier_freq}Hz)",
        "3. Salida del Mezclador (Antes del filtro)",
        "4. Mensaje Recuperado (Filtrado y Escalado)"
    ]

    fig = graficar_etapas(t, signals, titulos, sr)

    # Guardar audios temporalmente y preparar para descarga
    output_files = {}
    temp_audio_dir = "temp_audio"
    os.makedirs(temp_audio_dir, exist_ok=True)

    audio_info = [
        ('1_mensaje_original.wav', y),
        ('2_modulada_dsb_sc.wav', x_t),
        ('3_salida_mezclador.wav', y_mix),
        ('4_recuperada_final.wav', m_recovered)
    ]

    for filename, sig in audio_info:
        filepath = os.path.join(temp_audio_dir, filename)
        guardar_wav(filepath, sig, sr)
        output_files[filename] = filepath

    return fig, output_files


def dsp_simulation_page():
    st.header("📻 Simulación de Modulación y Demodulación DSB-SC")
    st.markdown("""
    Esta sección permite simular el proceso de Modulación y Demodulación de Banda Lateral Doble con Portadora Suprimida (DSB-SC).
    Podrás cargar un fragmento de audio, modularlo sobre una portadora de alta frecuencia y luego demodularlo para recuperar el mensaje original.
    Observa cómo la información de audio se desplaza en el dominio de la frecuencia durante la modulación y cómo se recupera mediante el proceso inverso.
    """)
    st.markdown("--->")

    st.write("Ajusta los parámetros para la simulación de modulación/demodulación DSB-SC:")

    # Parámetros configurables
    audio_filename_input = st.text_input("Nombre del archivo de audio (debe estar en /content/):", "Torero.mp3")
    start_time_s_input = st.slider("Tiempo de inicio (segundos):", 0, 300, 108)
    duration_s_input = st.slider("Duración del fragmento (segundos):", 1, 10, 5)
    carrier_freq_input = st.slider("Frecuencia de portadora (Hz):", 5000, 20000, 10000, step=1000)
    audio_bw_input = st.slider("Ancho de banda del audio (Hz):", 1000, 8000, 4000, step=500)

    if st.button("Ejecutar Simulación DSP"):
        fig_dsp, audio_files_dsp = procesar_dsp_streamlit(
            audio_filename_input,
            start_time_s_input,
            duration_s_input,
            carrier_freq_input,
            audio_bw_input
        )

        if fig_dsp:
            st.subheader("Resultados de la Simulación")
            st.pyplot(fig_dsp)
            plt.close(fig_dsp) # Close the figure to free memory

            st.subheader("Archivos de Audio Generados")
            for description, filepath in audio_files_dsp.items():
                with open(filepath, "rb") as file:
                    st.download_button(
                        label=f"Descargar {description.replace('_', ' ').replace('.wav', '')}",
                        data=file.read(),
                        file_name=description,
                        mime="audio/wav"
                    )
            st.success("Simulación DSP completada y archivos disponibles para descarga.")



def simular_caso(nombre, c_val, modo):
    global s

    # Define symbolic variables for m, c, k to be used in SymPy's TransferFunction
    m_sym, c_sym, k_sym = sym.symbols('m c k')

    # Numeric values for the current simulation case
    m_numeric = m_base
    c_numeric = c_val
    k_numeric_base = k_base # The base 'k' value

    # 1. FUNCIÓN DE TRANSFERENCIA
    if modo == "LA":
        num_sym_expr = 1
        # Define symbolic denominator expression
        den_sym_expr = m_sym*s**2 + c_sym*s + k_sym
        # G_sim uses numeric values for the 'control' library
        G_sim = ct.TransferFunction([1], [m_numeric, c_numeric, k_numeric_base])
        # G_sym uses symbolic expressions for SymPy's TransferFunction
        G_sym = TransferFunction(num_sym_expr, den_sym_expr, s)
        titulo = f"Lazo Abierto (LA)"
    else: # Modo Lazo Cerrado (LC)
        k_lc_numeric = k_numeric_base + 1
        num_sym_expr = 1
        # For LC, the 'k' in the denominator becomes 'k+1' in the standard form
        den_sym_expr = m_sym*s**2 + c_sym*s + (k_sym + 1)
        # G_sim uses numeric values for the 'control' library
        G_sim = ct.TransferFunction([1], [m_numeric, c_numeric, k_lc_numeric])
        # G_sym uses symbolic expressions for SymPy's TransferFunction
        G_sym = TransferFunction(num_sym_expr, den_sym_expr, s)
        titulo = f"Lazo Cerrado (LC)"

    st.subheader(f"{nombre} - {titulo}")

    # Define the substitution dictionary for SymPy plots. Maps symbols to current numeric values.
    mck_subs = {m_sym: m_numeric, c_sym: c_numeric}
    if modo == "LA":
        mck_subs[k_sym] = k_numeric_base
    else: # LC
        # For LC, k_sym is substituted with k_numeric_base, and the +1 is already in den_sym_expr
        mck_subs[k_sym] = k_numeric_base

    # --- CÁLCULO DE PARÁMETROS ---
    st.markdown("#### Parámetros Clave")

    damp_data = ct.damp(G_sim)

    try:
        zeta_calc = damp_data[0][0]
        wn_calc = damp_data[1][0]

        if zeta_calc > 1 and len(damp_data[0]) > 1:
            pass

    except IndexError:
        st.warning("No se pudo calcular ωn y ζ. Error al acceder a los datos de ct.damp().")
        return

    # Cálculos dinámicos (Usando los valores calculados con ct.damp())
    wd_calc = wn_calc * np.sqrt(1 - zeta_calc**2) if 0 < zeta_calc < 1 else 0
    Ts_calc = 4 / (zeta_calc * wn_calc) if zeta_calc > 0 else "∞"
    Tp_calc = np.pi / wd_calc if wd_calc > 0 else "N/A"

    # Polos del sistema
    polos = G_sim.poles()

    col1, col2, col3 = st.columns(3)
    col1.metric("Factor de Amortiguamiento (ζ)", f"{zeta_calc:.4f}")
    col2.metric("Frecuencia Natural (ωn)", f"{wn_calc:.4f} rad/s")
    col3.metric("Tiempo de Establecimiento (Ts)", f"{Ts_calc:.4f} s" if isinstance(Ts_calc, float) else Ts_calc)

    st.markdown(f"**Polos del Sistema:** {polos}")

    # --- GRÁFICAS ---
    st.markdown("#### Gráficas Dinámicas (usando `control` y `matplotlib`)")

    # Gráfica de Polos y Ceros (Usamos Matplotlib para Streamlit)
    fig_pz, ax_pz = plt.subplots(1, 1, figsize=(3, 2))
    if len(G_sim.zeros()) > 0:
        ax_pz.plot(np.real(G_sim.zeros()), np.imag(G_sim.zeros()), 'o', markersize=8, markeredgecolor='b', markerfacecolor='none', label='Ceros')
    ax_pz.plot(np.real(G_sim.poles()), np.imag(G_sim.poles()), 'x', markersize=8, markeredgecolor='r', markerfacecolor='r', label='Polos')
    ax_pz.axhline(0, color='black', linestyle='--', linewidth=0.5)
    ax_pz.axvline(0, color='black', linestyle='--', linewidth=0.5)
    ax_pz.set_title(f"Diagrama de Polos y Ceros ({nombre})", fontsize=8)
    ax_pz.set_xlabel(r"Eje Real ($\sigma$)", fontsize=7); ax_pz.set_ylabel(r"Eje Imaginario ($j\omega$)", fontsize=7)
    ax_pz.tick_params(axis='both', which='major', labelsize=6)
    ax_pz.grid(True)
    st.pyplot(fig_pz)
    plt.close(fig_pz)

    col4, col5 = st.columns(2)

    # Gráfica de Bode
    with col4:
        st.markdown("##### Diagrama de Bode")
        fig_bode = plt.figure(figsize=(3, 2))
        mag, phase, omega = ct.bode_plot(G_sim, plot=True, dB=True, omega_limits=(0.1*wn_calc, 10*wn_calc))
        plt.suptitle("")
        plt.tight_layout()
        st.pyplot(fig_bode)
        plt.close(fig_bode)


    # Gráfica de Respuestas Transitorias (Escalón, Impulso, Rampa)
    with col5:
        st.markdown("##### Respuestas Transitorias")
        t_final = float(Ts_calc) * 1.5 if isinstance(Ts_calc, float) else 5
        if t_final > 20: t_final = 20

        fig_resp, ax_resp = plt.subplots(1, 1, figsize=(3, 2))
        t_vec = np.linspace(0, t_final, 500)

        # Escalón (Step)
        t_s, y_s = ct.step_response(G_sim, t_vec)
        ax_resp.plot(t_s, y_s, label='Escalón')

        # Impulso (Impulse)
        t_i, y_i = ct.impulse_response(G_sim, t_vec)
        ax_resp.plot(t_i, y_i, label='Impulso')

        # Rampa (Ramp)
        rampa_input = t_vec
        t_r, y_r = ct.forced_response(G_sim, t_vec, rampa_input)
        ax_resp.plot(t_r, y_r, label='Rampa')

        ax_resp.set_title("Respuestas: Escalón, Impulso, Rampa", fontsize=8)
        ax_resp.set_xlabel("Tiempo (s)", fontsize=7); ax_resp.set_ylabel("Amplitud", fontsize=7)
        ax_resp.tick_params(axis='both', which='major', labelsize=6)
        ax_resp.grid(True); ax_resp.legend(fontsize=6)
        st.pyplot(fig_resp)
        plt.close(fig_resp)

    st.markdown("---")
    # --- SymPy Plots (new section) ---
    st.markdown("#### Gráficas Adicionales (usando SymPy y Matplotlib)")

    col_sympy_1, col_sympy_2, col_sympy_3 = st.columns(3)

    # Define t and w symbols for sympy.plot
    w = sym.Symbol('w')
    t = sym.Symbol('t')

    # Get H(jw) by substituting s=j*w in G_sym expression, then apply numeric substitutions
    Hjw_expr = G_sym.num.subs(s, sym.I*w) / G_sym.den.subs(s, sym.I*w)
    Hjw_expr_numeric_sub = Hjw_expr.subs(mck_subs)

    # 1. Time Response (h(t) - Step Response) using SymPy and Matplotlib
    with col_sympy_1:
        st.markdown("##### Respuesta al Escalón (SymPy)")
        try:
            # Step response in s-domain: G(s)/s
            G_step_s_expr = G_sym.num / (G_sym.den * s)

            # Substitute numeric values into the expression before inverse Laplace transform
            G_step_s_expr_numeric_sub = G_step_s_expr.subs(mck_subs)

            # Inverse Laplace Transform to get h(t)
            h_t_expr = inverse_laplace_transform(G_step_s_expr_numeric_sub, s, t)

            t_final_sympy = float(Ts_calc) * 1.5 if isinstance(Ts_calc, float) else 5
            if t_final_sympy > 20: t_final_sympy = 20

            # Convert symbolic expression to a numerical function and plot with matplotlib
            h_t_func = sym.lambdify(t, h_t_expr, 'numpy')
            t_vec_sympy = np.linspace(0, t_final_sympy, 500)

            fig_h_t, ax_h_t = plt.subplots(1, 1, figsize=(3, 2))
            ax_h_t.plot(t_vec_sympy, h_t_func(t_vec_sympy))
            ax_h_t.set_title('Respuesta al Escalón (SymPy)', fontsize=8)
            ax_h_t.set_xlabel('Tiempo', fontsize=7)
            ax_h_t.set_ylabel('Amplitud', fontsize=7)
            ax_h_t.tick_params(axis='both', which='major', labelsize=6)
            ax_h_t.grid(True)
            st.pyplot(fig_h_t)
            plt.close(fig_h_t)

        except Exception as e:
            st.error(f"No se pudo generar la gráfica de Respuesta al Escalón con SymPy: {e}. Puede que la expresión sea demasiado compleja para SymPy o que los parámetros no permitan una transformación simple.")

    # 2. Magnitude Plot using SymPy and Matplotlib
    with col_sympy_2:
        st.markdown("##### Magnitud de Bode (SymPy)")
        try:
            mag_func = sym.lambdify(w, abs(Hjw_expr_numeric_sub), 'numpy')
            w_vec_sympy = np.linspace(0.1, 10*wn_calc, 500) # Use a positive range for frequency

            fig_mag, ax_mag = plt.subplots(1, 1, figsize=(3, 2))
            ax_mag.plot(w_vec_sympy, mag_func(w_vec_sympy))
            ax_mag.set_title('Magnitud (SymPy)', fontsize=8)
            ax_mag.set_xlabel('Frecuencia', fontsize=7)
            ax_mag.set_ylabel('Magnitud', fontsize=7)
            ax_mag.tick_params(axis='both', which='major', labelsize=6)
            ax_mag.set_xscale('log') # Log scale for frequency axis is common in Bode plots
            ax_mag.grid(True, which="both", ls="-")
            st.pyplot(fig_mag)
            plt.close(fig_mag)
        except Exception as e:
            st.error(f"No se pudo generar la gráfica de Magnitud con SymPy: {e}")

    # 3. Phase Plot using SymPy and Matplotlib
    with col_sympy_3:
        st.markdown("##### Fase de Bode (SymPy)")
        try:
            phi_sympy_expr = sym.arg(Hjw_expr_numeric_sub)
            phase_func = sym.lambdify(w, phi_sympy_expr, 'numpy')
            w_vec_sympy = np.linspace(0.1, 10*wn_calc, 500) # Use a positive range for frequency

            fig_phase, ax_phase = plt.subplots(1, 1, figsize=(3, 2))
            ax_phase.plot(w_vec_sympy, phase_func(w_vec_sympy))
            ax_phase.set_title('Fase (SymPy)', fontsize=8)
            ax_phase.set_xlabel('Frecuencia', fontsize=7)
            ax_phase.set_ylabel('Fase', fontsize=7)
            ax_phase.tick_params(axis='both', which='major', labelsize=6)
            ax_phase.set_xscale('log') # Log scale for frequency axis is common in Bode plots
            ax_phase.grid(True, which="both", ls="-")
            st.pyplot(fig_phase)
            plt.close(fig_phase)
        except Exception as e:
            st.error(f"No se pudo generar la gráfica de Fase con SymPy: {e}")

    st.markdown("---")

    # --- ANÁLISIS DE LAS GRÁFICAS ---
    st.markdown("#### Análisis de Comportamiento")
    if zeta_calc < 1:
        st.markdown(f"Este sistema es **Subamortiguado** (ζ={zeta_calc:.2f} < 1). Esto se observa en los polos complejos conjugados en el Diagrama de Polos y Ceros y en la respuesta transitoria, que muestra oscilaciones antes de estabilizarse. El tiempo de establecimiento (Ts) es aproximadamente {Ts_calc:.2f} segundos, y el pico de sobreimpulso (Mp) y tiempo de pico (Tp) serían características importantes aquí, con Tp ~ {Tp_calc:.2f}s si es aplicable.")
    elif zeta_calc == 1:
        st.markdown(f"Este sistema es **Crítico** (ζ={zeta_calc:.2f} = 1). Presenta polos reales e idénticos. La respuesta transitoria es la más rápida posible sin oscilaciones, alcanzando el valor final en aproximadamente {Ts_calc:.2f} segundos. No hay sobreimpulso ni oscilaciones.")
    else:
        st.markdown(f"Este sistema es **Sobreamortiguado** (ζ={zeta_calc:.2f} > 1). Los polos son reales y distintos. La respuesta transitoria es lenta y sin oscilaciones, tomando aproximadamente {Ts_calc:.2f} segundos en establecerse. Es más lento que el sistema críticamente amortiguado.")

    st.markdown("---")

# --- SELECCIÓN EN BARRA LATERAL ---
st.sidebar.header("Opciones de Simulación")

seleccion_principal = st.sidebar.radio(
    "Selecciona una opción:",
    ("Introducción", "Simulación", "Simulación DSP")
)

if seleccion_principal == "Introducción":
    st.header("Bienvenido al Análisis del Sistema Masa-Resorte-Amortiguador")
    st.markdown("""
    Este dashboard interactivo te permite explorar el comportamiento dinámico de un sistema masa-resorte-amortiguador. Podrás analizar su respuesta tanto en **Lazo Abierto (LA)** como en **Lazo Cerrado (LC)**, y observar cómo diferentes niveles de amortiguamiento (subamortiguado, crítico, sobreamortiguado) afectan sus características clave y su respuesta a diversas entradas.

    **¿Qué es el Sistema Masa-Resorte-Amortiguador?**
    Es un modelo fundamental en ingeniería que describe el movimiento de una masa conectada a un resorte y un amortiguador. Este sistema se utiliza para estudiar vibraciones mecánicas y es la base para entender sistemas más complejos.

    **¿Cómo usar este Dashboard?**
    1. **Sección de Modelado:** Entenderás la base matemática del sistema.
    2. **Selecciona 'Simulación' en el menú de la izquierda.**
    3. **Elige el Modo de Lazo:** Lazo Abierto (LA) o Lazo Cerrado (LC).
    4. **Elige el Tipo de Amortiguamiento:** Subamortiguado, Crítico o Sobreamortiguado.

    Cada selección mostrará los parámetros clave, los diagramas de polos y ceros, las gráficas de Bode y las respuestas transitorias (escalón, impulso, rampa), junto con un análisis del comportamiento observado.
    """)
    st.markdown("---")

    st.header("Modelado Matemático y Parámetros Clave")
    st.markdown("El sistema masa-resorte-amortiguador se describe mediante la siguiente ecuación diferencial, basada en la Segunda Ley de Newton:")
    st.latex(r"m \frac{d^2y}{dt^2} + c \frac{dy}{dt} + k y = F_E(t)")
    st.markdown("Donde:")
    st.markdown("- $m$: masa (kg)")
    st.markdown("- $c$: constante de amortiguamiento (Ns/m)")
    st.markdown("- $k$: constante del resorte (N/m)")
    st.markdown("- $y(t)$: desplazamiento de la masa (m)")
    st.markdown("- $F_E(t)$: fuerza externa aplicada (N)")

    st.markdown("### Función de Transferencia (Lazo Abierto)")
    st.markdown("Aplicando la Transformada de Laplace (asumiendo condiciones iniciales cero) a la ecuación anterior, obtenemos la función de transferencia $G(s) = \frac{Y(s)}{F_E(s)}$:")
    st.latex(r"G(s) = \frac{1}{m s^2 + c s + k}")
    st.markdown("Para facilitar el análisis, esta función se normaliza a la forma estándar de un sistema de segundo orden:")
    st.latex(r"G(s) = \frac{\omega_n^2}{s^2 + 2\zeta\omega_n s + \omega_n^2}")

    st.markdown("### Parámetros Dinámicos Clave")
    st.markdown("De la forma normalizada, podemos identificar los siguientes parámetros:")

    st.markdown("*   **Frecuencia Natural no Amortiguada ($\omega_n$)**:")
    st.latex(r"\omega_n = \sqrt{\frac{k}{m}} \quad \text{(rad/s)}")

    st.markdown("*   **Factor de Amortiguamiento ($\zeta$)**:")
    st.latex(r"\zeta = \frac{c}{2 \sqrt{mk}} = \frac{c}{c_{crítico}}")
    st.markdown("Donde $c_{crítico} = 2 \sqrt{mk}$ es el amortiguamiento crítico. Este factor determina el tipo de respuesta del sistema:")
    st.markdown("-   **Subamortiguado ($\zeta < 1$)**: El sistema oscila antes de estabilizarse.")
    st.markdown("-   **Crítico ($\zeta = 1$)**: El sistema se estabiliza lo más rápido posible sin oscilaciones.")
    st.markdown("-   **Sobreamortiguado ($\zeta > 1$)**: El sistema se estabiliza lentamente sin oscilaciones.")

    st.markdown("*   **Frecuencia Natural Amortiguada ($\omega_d$)** (para sistemas subamortiguados):")
    st.latex(r"\omega_d = \omega_n \sqrt{1 - \zeta^2} \quad \text{(rad/s)}")

    st.markdown("*   **Tiempo de Establecimiento ($T_s$)** (criterio del 2%):")
    st.latex(r"T_s \approx \frac{4}{\zeta \omega_n} \quad \text{(s)}")

    st.markdown("*   **Tiempo de Pico ($T_p$)** (para sistemas subamortiguados):")
    st.latex(r"T_p = \frac{\pi}{\omega_d} \quad \text{(s)}")

    st.markdown("### Caso de Lazo Cerrado")
    st.markdown("En el modo de Lazo Cerrado con retroalimentación unitaria, la función de transferencia se modifica a:")
    st.latex(r"G_{LC}(s) = \frac{G(s)}{1 + G(s)}")
    st.markdown("Para el sistema masa-resorte-amortiguador, esto resulta en un cambio efectivo en la constante del resorte, de $k$ a $k+1$, lo que a su vez afecta la $\omega_n$ y el factor $\zeta$ del sistema en lazo cerrado.")


elif seleccion_principal == "Simulación": # seleccion_principal == "Simulación"
    st.header("Simulación Dinámica del Sistema")
    st.markdown("Utiliza las opciones del menú lateral para seleccionar el modo de lazo y el tipo de amortiguamiento.")
    st.markdown("---")

    modo_seleccionado = st.sidebar.radio(
        "Selecciona el Modo de Lazo:",
        ("Lazo Abierto (LA)", "Lazo Cerrado (LC)")
    )

    caso_seleccionado = st.sidebar.radio(
        "Selecciona el Tipo de Amortiguamiento:",
        CASOS.keys()
    )

    st.header(f"Simulación: {caso_seleccionado} en {modo_seleccionado}")

    # --- LLAMADA A LA FUNCIÓN DE SIMULACIÓN BASADA EN LAS SELECCIONES ---
    if modo_seleccionado == "Lazo Abierto (LA)":
        simular_caso(caso_seleccionado, CASOS[caso_seleccionado]['c'], "LA")
    else:
        simular_caso(caso_seleccionado, CASOS[caso_seleccionado]['c'], "LC")
elif seleccion_principal == "Simulación DSP":
    dsp_simulation_page()


Overwriting app.py


In [42]:
# --- Ejecución y Túnel (Celda 3 - FINAL) ---

# 1. Ejecuta el script app.py en segundo plano (nohup... &)
!nohup streamlit run app.py > output.log 2>&1 &
print("Streamlit iniciado en segundo plano.")

# 2. Instalar pyngrok y configurar el túnel
!pip install pyngrok -q
from pyngrok import ngrok
import time

# 3. AUTENTICACIÓN: Configura tu token de NGROK
# ¡REEMPLAZA <TU_TOKEN_AQUI> CON TU TOKEN REAL!
ngrok.set_auth_token("36J2cz3w13muhbuMkbdNK7XKNGG_5PWfovHKEpdyKn3zLgxWb")

# 4. Cierra cualquier túnel ngrok anterior y limpia la configuración
ngrok.kill()
print("Procesos ngrok antiguos terminados.")

# 5. Esperar un momento para asegurar que Streamlit haya iniciado completamente (opcional, pero útil)
time.sleep(5)

# 6. Abre el nuevo túnel en el puerto 8501
try:
    public_url = ngrok.connect(8501) # Se ha simplificado la llamada aquí
    print(f"\n🎉 Tu Dashboard de Streamlit está disponible en: {public_url}")
except Exception as e:
    print(f"\n❌ Error al conectar ngrok: {e}")
    print("Intenta revisar tu token de ngrok o reiniciar el entorno de Colab.")

Streamlit iniciado en segundo plano.
Procesos ngrok antiguos terminados.

🎉 Tu Dashboard de Streamlit está disponible en: NgrokTunnel: "https://anissa-inordinate-madilyn.ngrok-free.dev" -> "http://localhost:8501"
